In [1]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForCausalLM, Trainer, TrainingArguments
torch.cuda.empty_cache()

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = load_dataset("code_search_net", "python",cache_dir="./Datasets")

In [4]:
dataset.column_names

{'train': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'test': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'validation': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url']}

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base", cache_dir="./Models")
model = RobertaForCausalLM.from_pretrained("microsoft/codebert-base", cache_dir="./Models",is_decoder = True).to(device)

Some weights of RobertaForCausalLM were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize_code(code_snippet, tokenizer):
    return tokenizer(
        code_snippet, max_length=512, truncation=True, padding='max_length', return_tensors="pt"
    )

In [7]:
def preprocess_data(batch, tokenizer):
    batch["input_ids"] = tokenize_code(batch["func_code_string"], tokenizer)["input_ids"]
    batch["labels"] = tokenize_code(batch["func_documentation_string"], tokenizer)["input_ids"]
    return batch

In [8]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(5000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(1000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)

In [9]:
model

RobertaForCausalLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [10]:
training_args = TrainingArguments(
    output_dir="./codebert-finetuned-roberta/results",    # output directory
    num_train_epochs=15,                   # number of epochs
    per_device_train_batch_size=4,       # batch size
    per_device_eval_batch_size=4,        # eval batch size
    warmup_steps=1000,                     # warmup steps
    weight_decay=0.01,                    # weight decay
    logging_dir="./logs",                 # logging directory
    logging_steps=100,
    evaluation_strategy="steps",          # Evaluation after every logging step
    save_total_limit=2,                   # Keep only last two checkpoints
    save_steps=1000,                       # Save model every 500 steps
    report_to="none",
    fp16=True,
    learning_rate=5e-5                 # No reports (e.g., to wandb)
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\accelerate\accelerator.py:463: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

  0%|          | 0/18750 [00:00<?, ?it/s]We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  1%|          | 100/18750 [09:18<29:30:23,  5.70s/it]

{'loss': 13.1104, 'grad_norm': 35.545963287353516, 'learning_rate': 4.800000000000001e-06, 'epoch': 0.08}


                                                      
  1%|          | 100/18750 [16:53<29:30:23,  5.70s/it]

{'eval_loss': 5.102454662322998, 'eval_runtime': 454.5891, 'eval_samples_per_second': 2.2, 'eval_steps_per_second': 0.55, 'epoch': 0.08}


  1%|          | 200/18750 [26:38<29:56:45,  5.81s/it]  

...

{'loss': 1.131, 'grad_norm': 2.287902355194092, 'learning_rate': 3.9750000000000004e-05, 'epoch': 0.64}


                                                        
  4%|▍         | 800/18750 [2:18:23<29:00:26,  5.82s/it]

{'eval_loss': 1.3159384727478027, 'eval_runtime': 453.04, 'eval_samples_per_second': 2.207, 'eval_steps_per_second': 0.552, 'epoch': 0.64}


  5%|▍         | 900/18750 [2:28:19<30:08:21,  6.08s/it]  

{'loss': 1.2593, 'grad_norm': 0.7120000720024109, 'learning_rate': 4.4750000000000004e-05, 'epoch': 0.72}


In [ ]:
model.save_pretrained("./codebert-finetuned-roberta")
tokenizer.save_pretrained("./codebert-finetuned-roberta")

In [17]:
def generate_documentation(code_snippet, model, tokenizer):
    # Tokenize input
    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Generate output
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)
    
    # Decode generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [15]:
code_example = """
def add_numbers(a, b):
    return a + c
"""

In [ ]:
documentation = generate_documentation(code_example, model, tokenizer)
print("Generated Documentation:", documentation)

In [ ]:
model